# **Neighbourhood Consensus Networks**

**Authors: Ignacio Rocco†, Mircea Cimpoi‡, Relja Arandjelovic´
§, Akihiko Torii∗,
Tomas Pajdla‡, Josef Sivic†,‡**

**†Inria / ‡CIIRC, CTU in Prague / §DeepMind / ∗Tokyo Institute of Technology**

**[Official Github Code](https://github.com/ignacio-rocco/ncnet)** / **[Project Page](https://www.di.ens.fr/willow/research/ncnet/)** / **[Pdf](https://arxiv.org/abs/1810.10510)**

---

**Edited By Su Hyung Choi (Key Summary & Code Practice)**

If you have any issues on this scripts, please PR to the repository below.

**[Github: @JonyChoi - Computer Vision Paper Reviews](https://github.com/jonychoi/Computer-Vision-Paper-Reviews)**

Edited April 14 2022

---

## **Practice For Ncnet (Neighbourhood Consensus Network)**

This is a **code practice for the core part** of implementation of the paper "Neighbourhood Consensus Networks" by I. Rocco, M. Cimpoi, R. Arandjelović, A. Torii, T. Pajdla and J. Sivic.
For more information check out the project [website](https://www.di.ens.fr/willow/research/ncnet/) and the paper on [arXiv](https://arxiv.org/abs/1810.10510).

**Reference From [ignacio-rocco/ncnet](https://github.com/ignacio-rocco/ncnet)**

### **Index**

- Overview
- Part 1. Model Implementation
- Part 2. Dataset
- Part 3. Training

### **1. Overview**

![](https://camo.githubusercontent.com/e454c9717acea8b28287cf5b82a7c2a275302ec8634a620742311cdd22db87da/68747470733a2f2f7777772e64692e656e732e66722f77696c6c6f772f72657365617263682f6e636e65742f696d616765732f7465617365722e706e67)

### **Part 1. Model Implementation**

At this section, we are going to review about the Core of the Ncnet, the ***4D Convolutional Neural Network***, which enables the Ncnet injecting the ***Neighbourhood Consensus*** from the correlation volumes which is the aggregation of the 4 dimensional tensor from the pair of images 

You can find the directory of the original sources here ***[ignacio-rocco/ncnet/lib/](https://github.com/ignacio-rocco/ncnet/tree/master/lib)***

---

### **Part 2. Dataset**

### **Part 3. Training**